In [26]:
using SymEngine
using Expr2LaTeX
using MacroTools

In [2]:
matrix_sym(name, nrows, ncols) = [symbols(Symbol(name,:_,i,:_,j)) for i in 1:nrows, j in 1:ncols]

matrix_sym (generic function with 1 method)

In [3]:
"Converts a SymEngine expression into a julia one"
function juliaize(symexpr)
    str = repr(symexpr)
    str = replace(str, "SymEngine.Basic", "")
    str = replace(str, r"(.)_(\d+)_(\d+)", s"\g<1>[\g<2>, \g<3>]")
    parse(str)
end

juliaize

In [31]:
function matmul_sym(X1,X2,Y1,Y2)
    x = matrix_sym(:X, X1, X2)
    y = matrix_sym(:Y, Y1, Y2)
    x*y
end
matmul_sym(args...) = matmul_sym(Val.(args)...)

matmul(x,y) = matmul(x,y,Val.(size(x))..., Val.(size(y))...)

@generated function matmul(X, Y, ::Val{X1},::Val{X2}, ::Val{Y1}, ::Val{Y2}) where {X1,X2,Y1,Y2}
    r = juliaize(matmul_sym(X1,X2,Y1,Y2))
    quote
        @inbounds ret = $(r)
        ret
    end |> macroexpand |> rmlines |> unblock    
end

matmul (generic function with 2 methods)

In [32]:
x=[4 1]'
y = [1 5]
@code_lowered matmul(x,y,Val.(size(x))..., Val.(size(y))...)

CodeInfo(:(begin 
        nothing
        r = (Main.juliaize)((Main.matmul_sym)($(Expr(:static_parameter, 1)), $(Expr(:static_parameter, 2)), $(Expr(:static_parameter, 3)), $(Expr(:static_parameter, 4)))) # line 12:
        return (((Core._expr)(:block, $(Expr(:copyast, :($(QuoteNode(:( # In[31], line 13:)))))), (Core._expr)(:macrocall, Symbol("@inbounds"), (Core._expr)(:(=), :ret, r)), $(Expr(:copyast, :($(QuoteNode(:( # In[31], line 14:)))))), :ret) |> Main.macroexpand) |> Main.rmlines) |> Main.unblock
    end))

In [38]:
@time matmul(rand(3,400),rand(400,30))

  0.002504 seconds (36.10 k allocations: 953.656 KiB)


3×30 Array{Float64,2}:
  95.9903   99.0419   93.2743  89.4836  …   99.076  102.706   96.3882
 100.949   106.858   104.919   98.4202     105.524  107.544  104.892 
 101.068   102.967    97.6117  92.5801      99.457  103.826   99.0382

In [39]:
@time (rand(3,40)*rand(40,30))

  0.000034 seconds (7 allocations: 11.516 KiB)


3×30 Array{Float64,2}:
 11.8048   11.9445  11.7501   10.4895   …  10.3672   11.1396   12.4505
 11.298    11.6671  10.9381   10.1879       9.53003  10.9246   11.0608
  9.38636  11.0255   9.54885   9.06496      9.05131   9.60012  11.2243